In [1]:
import ROOT
import podio

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x46bdde0


In [2]:
%jsroot on

In [3]:
# ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.ildStyle.SetOptStat(1)
ROOT.ildStyle.SetPalette(ROOT.kBird)

In [5]:
%%cpp
using namespace ROOT::VecOps;

In [6]:
ROOT.gInterpreter.Declare("#include <podio/DataSource.h>")
ROOT.gInterpreter.Declare("#include <edm4hep/utils/dataframe.h>")
ROOT.gInterpreter.Declare("#include <edm4hep/utils/kinematics.h>")
ROOT.gInterpreter.Declare("#include <edm4hep/utils/vector_utils.h>")
ROOT.gSystem.Load("libedm4hep")
_edm  = ROOT.edm4hep.ReconstructedParticleData()

In [7]:
# df = ROOT.RDataFrame("events", "data/truejet/test_dst/eLpR_15176_0.truejet.edm4hep.root")
df = ROOT.podio.CreateDataFrame("data/truejet/test_dst/eLpR_15176_0.truejet.edm4hep.root")

In [8]:
# What outputs do I want?
# Gen:
#   true collision 4 vector
#   ISR
#   final colour neutrals:
#   l, nu, l FSR
#   j1, j2, and their FSR (extra gluon or whatever)?
# Reco:
# ...

# but for now lets do the naive and quick thing to see where stuff goes wrong
# e.g. ignore truejet
# return on gen level leading e, nu? maybe cut on the leading e having to be inside the detector? I guess there will be large e in the overlay also
# and everything else?

# can not access cross section from podio data source so use a second data frame to write out the weights for stage2

In [9]:
# for now just look at some overlay urgh but not even that is easy as that is a subset collection and annoying to access


In [10]:
df.Describe()

Dataframe from datasource Custom Datasource

Property                Value
--------                -----
Columns in total          106
Columns from defines        0
Event loops run             0
Processing slots            1

Column                                          Type                                            Origin
------                                          ----                                            ------
BCalClusters                                    edm4hep::ClusterCollection                      Dataset
BCalRecoParticle                                edm4hep::ReconstructedParticleCollection        Dataset
BCalRecoParticle_startVertices                  edm4hep::VertexRecoParticleLinkCollection       Dataset
BuildUpVertex                                   edm4hep::VertexCollection                       Dataset
BuildUpVertex_RP                                edm4hep::ReconstructedParticleCollection        Dataset
BuildUpVertex_RP_startVertices                  

In [11]:
%%cpp
ROOT::Math::PxPyPzEVector make_PxPyPzEVec(edm4hep::Vector3f P, float E)
{
    return ROOT::Math::PxPyPzEVector(P.x, P.y, P.z, E);
}

In [12]:
df = df.Define("overlay_esum", "Sum(RVec(PFOsFromOverlay.energy()))")

df = df.Define("PFOsFromOverlay_lvecs", "Map(RVec(PFOsFromOverlay.momentum()), RVec(PFOsFromOverlay.energy()), make_PxPyPzEVec)")
df = df.Define("PFOsFromOverlay_theta", "return Map(PFOsFromOverlay_lvecs, [] (const auto& el) {return el.Theta();});")
df = df.Define("PFOsFromOverlay_p", "return Map(PFOsFromOverlay_lvecs, [] (const auto& el) {return el.P();});")
df = df.Define("PFOsFromOverlay_pt", "return Map(PFOsFromOverlay_lvecs, [] (const auto& el) {return el.Pt();});")

df = df.Define("PFOsWithoutMCOverlay_lvecs", "Map(RVec(PFOsWithoutMCOverlay.momentum()), RVec(PFOsWithoutMCOverlay.energy()), make_PxPyPzEVec)")
df = df.Define("PFOsWithoutMCOverlay_theta", "return Map(PFOsWithoutMCOverlay_lvecs, [] (const auto& el) {return el.Theta();});")
df = df.Define("PFOsWithoutMCOverlay_p", "return Map(PFOsWithoutMCOverlay_lvecs, [] (const auto& el) {return el.P();});")
df = df.Define("PFOsWithoutMCOverlay_pt", "return Map(PFOsWithoutMCOverlay_lvecs, [] (const auto& el) {return el.Pt();});")

In [13]:
h_overlay_esum = df.Histo1D("overlay_esum")

h_overlay_p = df.Histo1D("PFOsFromOverlay_p")
h_overlay_pt = df.Histo1D(("", ";p_{T} [GeV]", 150, 0., 150.), "PFOsFromOverlay_pt")
h_overlay_theta = df.Histo1D("PFOsFromOverlay_theta")
h_2d_overlay_theta_p = df.Histo2D(("", ";#theta; p", 50, 0., 3.14156, 50, 0., 15.), "PFOsFromOverlay_theta", "PFOsFromOverlay_p")
h_2d_overlay_pt_p = df.Histo2D(("", ";p_{T} [GeV]; p [GeV]", 20, 0., 10., 150, 0., 150.), "PFOsFromOverlay_pt", "PFOsFromOverlay_p")

h_nonoverlay_p = df.Histo1D("PFOsWithoutMCOverlay_p")
h_nonoverlay_pt = df.Histo1D(("", ";p_{T} [GeV]", 150, 0., 150.), "PFOsWithoutMCOverlay_pt")
h_nonoverlay_theta = df.Histo1D("PFOsWithoutMCOverlay_theta")
h_2d_nonoverlay_theta_p = df.Histo2D(("", ";#theta; p", 50, 0., 3.14156, 50, 0., 15.), "PFOsWithoutMCOverlay_theta", "PFOsWithoutMCOverlay_p")
h_2d_nonoverlay_pt_p = df.Histo2D(("", ";p_{T} [GeV]; p [GeV]", 20, 0., 10., 150, 0., 150.), "PFOsWithoutMCOverlay_pt", "PFOsWithoutMCOverlay_p")

In [14]:
# c_overlay_esum_norm = ROOT.TCanvas()
# h_overlay_esum_norm = h_overlay_esum.Clone()
# h_overlay_esum_norm.Scale(1 / h_overlay_esum.Integral())
# h_overlay_esum_norm.Draw()
# c_overlay_esum_norm.Draw()

# c_overlay_esum_norm_cum = ROOT.TCanvas()
# # h_overlay_esum_norm_cum = h_overlay_esum_norm.GetCumulative(ROOT.kTRUE)
# h_overlay_esum_norm_cum = h_overlay_esum_norm.GetCumulative(ROOT.kFALSE)
# h_overlay_esum_norm_cum.Draw()
# c_overlay_esum_norm_cum.Draw()

c_overlay_p = ROOT.TCanvas()
h_overlay_p.Draw()
c_overlay_p.Draw()

c_overlay_pt = ROOT.TCanvas()
h_overlay_pt.Draw()
c_overlay_pt.Draw()

c_2d_overlay_pt_p = ROOT.TCanvas()
h_2d_overlay_pt_p.Draw()
c_2d_overlay_pt_p.Draw()

c_nonoverlay_p = ROOT.TCanvas()
h_nonoverlay_p.Draw()
c_nonoverlay_p.Draw()

c_nonoverlay_pt = ROOT.TCanvas()
h_nonoverlay_pt.Draw()
c_nonoverlay_pt.Draw()

c_2d_nonoverlay_pt_p = ROOT.TCanvas()
h_2d_nonoverlay_pt_p.Draw()
c_2d_nonoverlay_pt_p.Draw()